In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
from tkcalendar import DateEntry
from tkinter import messagebox
import pymysql
from plyer import notification
from datetime import datetime
import time


class desktop:
    def __init__(self,root):
        self.root = root
        icon_path = "C:/Users/bhask/Downloads/window1.ico"
        self.root.iconbitmap(icon_path)
        self.root.title("Capstone")
        root.configure(background='#89B5FE')
        self.root.geometry("1350x700+100+50")
        
        title = Label(self.root, text = 'ZenFlow',bd=8,relief=GROOVE,font=("times new roman",30),bg="#0087FF",fg="white")
        title.pack(side=TOP,fill=X)
        
        #-----All Variables-----
        self.date = StringVar()
        self.hour = StringVar()
        self.minute = StringVar()
        self.second = StringVar()
        self.tittle = StringVar()
        
        
        
        #----Notification----
        noti_frame = Frame(self.root,bd=5,relief=GROOVE,bg="#A2D1FC")
        noti_frame.place(x=50,y=80,width=600,height=600)
        
        noti_tittle = Label(noti_frame, text = 'Notification',bd=5,relief=GROOVE,font=("times new roman",20),bg="#0087FF",fg="white")
        noti_tittle.place(x=200,y=15)
        
        noti_date = Label(noti_frame, text = 'Select Date',font=("times new roman",18,"bold"),bg="#A2D1FC",fg="white")
        noti_date.place(x=70,y=100)
        
        date_entry = DateEntry(noti_frame,date_pattern='yyyy-mm-dd',textvariable=self.date,width=12, bg='#A2D1FC', fg='white', bd=2,relief=GROOVE)
        date_entry.place(x=300,y=100,width=80,height=30)
        
        noti_time = Label(noti_frame, text = 'Select Time',font=("times new roman",18,"bold"),bg="#A2D1FC",fg="white")
        noti_time.place(x=70,y=200)
        
        hour_spinbox = tk.Spinbox(noti_frame,textvariable=self.hour, from_=0, to=23, width=3)
        hour_spinbox.place(x=300,y=200,width=30,height=25)
        minute_spinbox = tk.Spinbox(noti_frame,textvariable=self.minute, from_=0, to=59, width=3)
        minute_spinbox.place(x=335,y=200,width=30,height=25)
        second_spinbox = tk.Spinbox(noti_frame,textvariable=self.second, from_=0, to=59, width=3)
        second_spinbox.place(x=370,y=200,width=30,height=25)
        
        
        noti_title = Label(noti_frame, text = 'Notification Title',font=("times new roman",18,"bold"),bg="#A2D1FC",fg="white")
        noti_title.place(x=70,y=300)
        
        title_entry = Entry(noti_frame,textvariable=self.tittle,font=("times new roman",15,"bold"), bd=2,relief=GROOVE)
        title_entry.place(x=300,y=300,width=180,height=30)
        
        noti_message = Label(noti_frame, text = 'Notification message',font=("times new roman",18,"bold"),bg="#A2D1FC",fg="white")
        noti_message.place(x=70,y=400)
        
        self.message_entry = Text(noti_frame,font=("times new roman",10), bd=2,relief=GROOVE)
        self.message_entry.place(x=300,y=400,width=230,height=80)
        
        #----Button Frame----
        
        btn_frame = Frame(noti_frame,bd=4,relief=RIDGE,bg="#A2D1FC")
        btn_frame.place(x=100,y=515,width=408,height=50)
        
        save_btn = Button(btn_frame,text='SAVE',width=10,command=self.save_noti).grid(row=0,column=0,padx=10,pady=7)
        update_btn = Button(btn_frame,text='UPDATE',width=10,command=self.update_detail).grid(row=0,column=1,padx=10,pady=7)
        delete_btn = Button(btn_frame,text='DELETE',width=10,command=self.delete_detail).grid(row=0,column=2,padx=10,pady=7)
        clear_btn = Button(btn_frame,text='CLEAR',width=10,command=self.clear_detail).grid(row=0,column=3,padx=10,pady=7)
        
    
        
        #----Detail----------
        detail_frame = Frame(self.root,bd=5,relief=GROOVE,bg="#A2D1FC")
        detail_frame.place(x=700,y=80,width=600,height=600)
        
        detail_tittle = Label(detail_frame, text = 'DETAILS',bd=5,relief=GROOVE,font=("times new roman",20),bg="#0087FF",fg="white")
        detail_tittle.place(x=230,y=15)
        
        #----Table---------
        table_frame = Frame(detail_frame,bd=5,relief=GROOVE,bg="white")
        table_frame.place(x=20,y=80,width=550,height=500)
        
        scroll_x = Scrollbar(table_frame, orient=HORIZONTAL)
        scroll_y = Scrollbar(table_frame, orient=VERTICAL)

        self.detail_table=ttk.Treeview(table_frame,columns=('Date','Time','Tittle','Message'), xscrollcommand=scroll_x.set, yscrollcommand=scroll_y.set)

        scroll_x.pack(side=BOTTOM, fill=X)
        scroll_y.pack(side=RIGHT, fill=Y)
        scroll_x.config(command=self.detail_table.xview) 
        scroll_y.config(command=self.detail_table.yview)

        
        self.detail_table.heading("Date", text="Date") 
        self.detail_table.heading("Time", text="Time")
        self.detail_table.heading("Tittle", text="Tittle")
        self.detail_table.heading("Message", text="Message")

        self.detail_table['show'] = 'headings'
        
        self.detail_table.column('Date',width=100)
        self.detail_table.column('Time',width=100)
        self.detail_table.column('Tittle',width=100)
        self.detail_table.column('Message',width=100)
        
        self.detail_table.pack(fill=BOTH,expand=1)
        self.detail_table.bind("<ButtonRelease-1>",self.get_cursor)
        self.display_detail()
        
        
    def save_noti(self):
        
        hrs = self.hour.get()
        mns = self.minute.get()
        sec = self.second.get()
        time = f"{hrs}:{mns}:{sec}"
                
        
        
        con = pymysql.connect(host='localhost',user='root',password='',database='capstone')
        cur = con.cursor()
        cur.execute("insert into notification values(%s,%s,%s,%s)",(self.date.get(),time,self.tittle.get(),self.message_entry.get('1.0',END)))
        con.commit()
        self.display_detail()
        self.clear_detail()
        con.close()
        messagebox.showinfo('Success','Notification has been Added!')
        
        
    def display_detail(self):
        con = pymysql.connect(host='localhost',user='root',password='',database='capstone')
        cur = con.cursor()
        cur.execute("select * from notification")
        rows = cur.fetchall()
        if len(rows)!=0:
            self.detail_table.delete(*self.detail_table.get_children())
            for row in rows:
                self.detail_table.insert('',END,values=row)
            con.commit()
        con.close()
        
        
    def clear_detail(self):
        self.date.set('')
        self.hour.set('')
        self.minute.set('')
        self.second.set('')
        self.tittle.set('')
        self.message_entry.delete('1.0',END)
        
        
    def get_cursor(self,evs):
        cursor_row = self.detail_table.focus()
        contents = self.detail_table.item(cursor_row)
        row = contents['values']
        self.date.set(row[0])
        time = row[1].split(':')
        self.hour.set(time[0])
        self.minute.set(time[1])
        self.second.set(time[2])
        self.tittle.set(row[2])
        self.message_entry.delete('1.0',END)
        self.message_entry.insert(END,row[3])
        
    def update_detail(self):
        hrs = self.hour.get()
        mns = self.minute.get()
        sec = self.second.get()
        time = f"{hrs}:{mns}:{sec}"
                
        con = pymysql.connect(host='localhost',user='root',password='',database='capstone')
        cur = con.cursor()
        cur.execute("update notification set date=%s , time=%s ,  message=%s where tittle=%s",(self.date.get(),
                                                                                                         time,
                                                                                                         self.message_entry.get('1.0',END),
                                                                                                         self.tittle.get()
                                                                                                        ))
        con.commit()
        self.display_detail()
        self.clear_detail()
        con.close()
        messagebox.showinfo('Success','Notification has been Updated!')
        
    def delete_detail(self):
        con = pymysql.connect(host='localhost',user='root',password='',database='capstone')
        cur = con.cursor()
        cur.execute("delete from notification where tittle=%s",self.tittle.get())
        con.commit()
        self.display_detail()
        self.clear_detail()
        con.close()
        messagebox.showinfo('Success','Notification has been Deleted!')
        



root = Tk()
obj = desktop(root)
root.mainloop()


    


